In [1]:
import pandas as pd
import numpy as np
import json

input_data = np.load(open('./data/train_input.npy', 'rb'))
out_data = np.load(open('./data/train_label.npy', 'rb'))

prepro_config=None

with open('./data/data_configs.json','r') as f :
    prepro_config = json.load(f)

In [2]:
#데이터 나누기
from sklearn.model_selection import train_test_split

RANDOM_SEED=13371447
TEST_SPLIT=0.1

X_train, X_test, y_train, y_test = train_test_split(input_data, out_data, test_size=TEST_SPLIT,
                                                   random_state=RANDOM_SEED)

In [3]:
import tensorflow as tf

In [4]:
#데이터 입력 함수
BATCH_SIZE = 16
NUM_EPOCHS = 3

def mapping_fn(X,Y):
    inputs, labels = {'x':X}, Y
    return inputs, labels

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) 
    #둘을 묶어 조각으로 만듬
    dataset = dataset.shuffle(buffer_size=len(X_train))
    #임의의 순서대로 데이터가 나옴, 데이터 전체길이를 넣어주면됨
    dataset = dataset.batch(BATCH_SIZE)
    #배치사이즈를 지정함으로 써 한번에 16개씩 나옴
    dataset = dataset.map(mapping_fn)
    #하나의 딕셔너리로 묶기위해서
    dataset = dataset.repeat(count=NUM_EPOCHS)
    #데이터를 반복해서 총 3번 불러옴
    iterator = dataset.make_one_shot_iterator()
    #데이터를 하나씩 사용할 수 있게 함
    
    return iterator.get_next() # 데이터가 하나씩 나옴
    
    
    
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))
    dataset = dataset.map(mapping_fn)
    dataset = dataset.batch(len(X_test))
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [5]:
#모델 하이퍼파라미터 정의
VOCAB_SIZE = prepro_config['vocab_size']
WORD_EMBEDDING_DIM = 100
HIDDEN_STATE_DIM = 150
DENSE_FEATURE_DIM = 150

learning_rate = 0.001  

In [6]:

def model_fn(features, labels, mode):
    
    #모델 구현
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    embedding_layer = tf.keras.layers.Embedding(VOCAB_SIZE, WORD_EMBEDDING_DIM)(features['x'])
    #features['x']는 파이썬 딕셔너리 형태
    
    embedding_layer = tf.keras.layers.Dropout(0.2)(embedding_layer)
    
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [HIDDEN_STATE_DIM, HIDDEN_STATE_DIM]]
    #심층 순환 신경망 모델은 LSTM이며 이를 객체화 함, Cell객체 여러개 생성해 리스트로 만들어 주며 hidden state의 차원만 정의
    #LSTMCell을 쌓게 되면 이를 하나의 MultiRNN으로 묶음-> wrapping
    
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    #MultiRNNCell을 생성해 LSTM신경망을 구현
    
    #네트워크와 임베딩 벡터를 연산하기위해 dynamic_rnn 함수 선언
    #이는 for문없이 자동으로 순환 신경망을 만들어 주는 역할
    #cell=순환신경망객체 , input=입력값(임베딩레이어) , dtype=데이터타입 설정
    outputs, state = tf.nn.dynamic_rnn(cell = multi_rnn_cell,
                                      inputs = embedding_layer,
                                      dtype = tf.float32)
    
    
    outputs = tf.keras.layers.Dropout(0.2)(outputs)
    #출력값을 Dense에 적용
    #LSTM신경망의 마지막 출력값에 넣어준다
    #출력값에 [:,-1,:]로 마지막 값만 뽑은 후 적용
    #활성화 함수는 하이퍼블릭 탄젠트(hyperbolic tangent)
    hidden_layer = tf.keras.layers.Dense(DENSE_FEATURE_DIM, activation=tf.nn.tanh)(outputs[:,-1,:])
    hidden_layer = tf.keras.layers.Dropout(0.2)(hidden_layer)
    
    #최종적으로 1dim 의 output만 할 수 있도록 dense layer를 활용해 차원 변환
    #감정이 긍정인지 부정인지 판단 할 수 있도록 출력값을 하나로 만듬, 입력 벡터에 대한 차원 수 를 바꿈
    #이 역할을 Dense가 함
    logits = tf.keras.layers.Dense(1)(hidden_layer) #logit은 출력값
    logits = tf.squeeze(logits, axis=-1)
    
    
    #모델 예측을 위한 구현
    #모델 출력값을 0~1 사이값으로 정의하기 위해 sigmoid함수 활용
    if PREDICT:
        pred = tf.nn.sigmoid(logits)
        return tf.estimator.EstimatorSpec(mode=mode, predictions={'sentiment':tf.round(pred)}) #예측값 입력
    
    
    #모델 학습을 위한 구현
    #모델에서 구현한 값과 정답 라벨을 가지고 손실(loss)값을 구해 Adam Optimizer을 활용해 모델 파라미터 최적화
    #로스값은 logits과 정답인 labels변수를 가지고 구함
    #logits값이 스케일을 맞춰 두지 않았지만 sigmoid_cross_entropy를 통해 손실값을 구할 수 있음
    loss = tf.losses.sigmoid_cross_entropy(labels, logits)
    
    #로스값 구한 후 파라미터를 최적화하고자 경사도 하강법을 진행한다
    #경사도 하강법인 AdamOptimizer객체를 생성할때 학습률을 입력, 또한 global_step값을 넣어야한다.
    if TRAIN:
        global_step = tf.train.get_global_step() #현재 global_step값 얻음
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(mode = mode, train_op = train_op, loss=loss) 
    #estimatorSpec에서 학습 연산과 손실값이 필요, 각 모드에 따라 필요한 입력값 다름
    #학습의 경우 학습연산과 손실값 필요
        
    #모델 평가를 위한 구현
    #감정 예측값이 정답 라벨과 얼마나 일치하는지 정확도를 봄
    if EVAL:
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))#정확도
        eval_metric_ops = {'acc':accuracy}
        
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops) #평가 연산 입력

In [7]:
est = tf.estimator.Estimator(model_fn, model_dir='./checkpoint/rnn')
#est객체 생성시 인자로 작성한 모델 함수 이름을 입력한다.
#생성한 객체를 통해 학습과 평가, 예측 등의 실행이 가능하다.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './checkpoint/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002A2FE9BD2C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
est.train(train_input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Inst

INFO:tensorflow:global_step/sec: 5.01766
INFO:tensorflow:loss = 0.0014738203, step = 17682 (19.931 sec)
INFO:tensorflow:global_step/sec: 5.02497
INFO:tensorflow:loss = 0.0014763576, step = 17782 (19.899 sec)
INFO:tensorflow:global_step/sec: 5.03736
INFO:tensorflow:loss = 0.0004722542, step = 17882 (19.852 sec)
INFO:tensorflow:global_step/sec: 5.03356
INFO:tensorflow:loss = 0.16078313, step = 17982 (19.867 sec)
INFO:tensorflow:global_step/sec: 5.02875
INFO:tensorflow:loss = 0.0005613648, step = 18082 (19.886 sec)
INFO:tensorflow:global_step/sec: 5.03407
INFO:tensorflow:loss = 0.0061447457, step = 18182 (19.866 sec)
INFO:tensorflow:global_step/sec: 5.03002
INFO:tensorflow:loss = 0.00035396143, step = 18282 (19.881 sec)
INFO:tensorflow:Saving checkpoints for 18302 into ./checkpoint/rnn\model.ckpt.
INFO:tensorflow:Loss for final step: 0.058712885.


In [9]:
est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-24T04:33:08Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/rnn\model.ckpt-18302
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-04-24-04:33:12
INFO:tensorflow:Saving dict for global step 18302: acc = 0.8576, global_step = 18302, loss = 0.7935023
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18302: ./checkpoint/rnn\model.ckpt-18302


{'acc': 0.8576, 'loss': 0.7935023, 'global_step': 18302}

In [10]:
test_input_data = np.load(open('./data/test_input.npy','rb'))
test_id = np.load(open('./data/test_id.npy','rb'),allow_pickle=True)

In [11]:
def mapping_fn2(X):
    inputs = {'x':X}
    return inputs

def test_input_fn2():
    dataset = tf.data.Dataset.from_tensor_slices((test_input_data))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn2)
    
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [12]:
prediction=np.array([p['sentiment'] for p in est.predict(test_input_fn2)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/rnn\model.ckpt-18302
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [13]:
prediction

array([1., 0., 0., ..., 0., 1., 0.], dtype=float32)

In [14]:
#제출하기위한 파일 생성
output=pd.DataFrame(data={'id':test_id, 'sentiment':list(prediction)})
output.to_csv('./data/rnn_predic.csv',index=False,quoting=3)